# 1. Setup

## 1.1 Installing Libraries

Reference: [Llama Index Installation and Setup](https://docs.llamaindex.ai/en/stable/getting_started/installation/)

In [ ]:
!pip install python-dotenv llama-index chromadb llama-index-vector-stores-chroma llama-index-retrievers-bm25 EbookLib html2text langchain-text-splitters

## 1.2 Importing Libraries

In [3]:
import chromadb

from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, PromptTemplate, get_response_synthesizer
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline
from llama_index.llms.openai import OpenAI
from llama_index.llms.groq import Groq
from llama_index.core.retrievers import QueryFusionRetriever

from ebooklib import epub
import uuid
import os
from pathlib import Path
from dotenv import load_dotenv
import nest_asyncio
from enum import Enum

nest_asyncio.apply()

e:\Capelin\llama-index-test\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


## 1.3 Importing Environment Variables

In [4]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

## 1.4 Setting up Embedding Model

In [8]:
embed_model = OpenAIEmbedding(model_name="text-embedding-ada-002", api_key=OPENAI_API_KEY)

## 1.5 Setting up LLM

In [5]:
llm_openai = OpenAI(api_key=OPENAI_API_KEY, model_name="gpt-4o-mini", temperature=0.1)
llm_groq = Groq(api_key=GROQ_API_KEY, model="deepseek-r1-distill-llama-70b", temperature=0.1)

# 2. Setting up IndexedVectorStore

In [ ]:
# The name "IndexedVectorStore" emphasizes that the class handles both the vector store and the index

class IndexedVectorStore:
    def __init__(self):
        self.db = chromadb.PersistentClient(path="../the-server/vectorstore/")
        self.chroma_collection = self.db.get_or_create_collection("transcription_project")
        self.vector_store = ChromaVectorStore(chroma_collection=self.chroma_collection)
        self.index = VectorStoreIndex.from_vector_store(
            self.vector_store,
            embed_model=embed_model,
        )

    def add_documents(self, documents: list) -> None:
        # Add the documents to the LlamaIndex and persist them
        for document in documents:
            self.index.insert(document)
        self.index.storage_context.persist(persist_dir="./db")

In [9]:
vectorstore = IndexedVectorStore()

# 3. Loading Data from Directory using `SimpleDirectoryReader`

Reference: [Loaders](https://docs.llamaindex.ai/en/stable/understanding/loading/loading/)

Extracting Metadata Reference: [SimpleDirectoryReader](https://docs.llamaindex.ai/en/stable/module_guides/loading/simpledirectoryreader/)

We can specify a function that will read each file and extract metadata that gets attached to the resulting Document object for each file by passing the function as `file_metadata`

In [10]:
def extract_epub_metadata(book_path: str) -> dict:
    book_path = Path(book_path)
    if not book_path.exists():
        raise FileNotFoundError(f"EPUB file not found at path: {book_path}")
    book = epub.read_epub(str(book_path))

    return {
        "id": f"epub-{uuid.uuid4().hex}",
        "title": book.get_metadata("DC", "title")[0][0].rstrip(".epub") if book.get_metadata("DC", "title") else "N/A",
        "author": book.get_metadata("DC", "creator")[0][0] if book.get_metadata("DC", "creator") else "",
        "language": book.get_metadata("DC", "language")[0][0] if book.get_metadata("DC", "language") else "",
        "description": book.get_metadata("DC", "description")[0][0] if book.get_metadata("DC", "description") else "",
        "type": "epub",
        "embeddings": "openaiembeddings"
    }

In [11]:
documents = SimpleDirectoryReader(input_dir="./data", file_metadata=extract_epub_metadata).load_data()

e:\Capelin\llama-index-test\.venv\Lib\site-packages\ebooklib\epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')
e:\Capelin\llama-index-test\.venv\Lib\site-packages\ebooklib\epub.py:1423: FutureWarning: This search incorrectly ignores the root element, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/xmlns:rootfile[@media-type]'
  for root_file in tree.findall('//xmlns:rootfile[@media-type]', namespaces={'xmlns': NAMESPACES['CONTAINERNS']}):


In [12]:
print(f"Total Documents: {len(documents)}")

Total Documents: 2


In [13]:
print(documents[0].metadata)

{'id': 'epub-3ac8f71491944cc988dc31160047ae61', 'title': 'Prophet Muhammad (SAW) ', 'author': 'Sakina Hasan Askari - XKP', 'language': 'en', 'description': 'The best of creation, the noblest example, the Mercy for all mankind, the Holy Prophet of Islam is the last Messenger from Allah. Find out about him and his teachings.\n-\nIslamicMobility.com', 'type': 'epub', 'embeddings': 'openaiembeddings'}


Loading a new book

In [9]:
new_book = SimpleDirectoryReader(input_files=["./data/pdf/custom_splitter.pdf"], file_metadata=extract_epub_metadata).load_data()
print(f"Metadata of first element: {new_book[0].metadata}")

# This way, we can load a new book and can use the same VectorStore object to add the new book to the index

Metadata of first element: {'page_label': '1', 'file_name': 'custom_splitter.pdf', 'type': 'pdf'}


In [12]:
len(new_book)

2070

In [11]:
new_book[0].text

'********  \nCHUNK 1  \n********  \n \n**Sub Title:**  \nAccording to the Fatawa of Ayatullah al Uzma Sayyid Ali al -Husaini Seestani  \n \nLaws on cleanliness, prayers, fasting, hajj, transactions, marriage, and other  \ntopics. According to the Risalah of Ayatullah Ali al -Husayni Al -Seestani.  \n \nGet PDF [3] Get EPUB [4] Get MOBI [5]  \n \n# Important Note  \n \nThe * sign after a number denotes that there is a total or partial variation  \nfrom the fatwa of Marhum Ayatullah Al Uzama Syed Abul Qasim Al Khu’i. These  \nlaws are also available online at Al -Islam.org.  \n \n# Taqlid: Following a Mujtahid  \n \n \n--------------------------------------------------  \n********  \nCHUNK 2  \n********  \n \n**Issue 1:**  \n* It is necessary for a Muslim to believe in the fundamentals of  \nfaith with his own insight and understanding, and he cannot follow anyone in  '

In [14]:
new_book[1].metadata

{'page_label': '2', 'file_name': 'custom_splitter.pdf', 'type': 'pdf'}

In [15]:
new_book[1].text

"this respect i.e. he cannot accept the word of another who knows, simply  \nbecause he has said it. However, one who has faith in the true tenets of  \nIslam, and manifests it by his deeds, is a Muslim and Mo'min, even if he is  \nnot very profound, and the laws related to a Muslim will hold good for him. In  \nmatters of religious laws, apart from the ones clearly defined, or ones which  \nare indisputable, a person must:  \n \n• either be a Mujtahid (jurist)  \n \n \n--------------------------------------------------  \n********  \nCHUNK 3  \n********  \n \n**Issue 2:**  \nTaqlid in religious laws means acting according to the verdict of  \na Mujtahid. It is necessary for the Mujtahid who is followed, to be male,  \nShi'ah Ithna Ash'ari, adult, sane, of legitimate birth, living and just  \n('Adil). A person is said to be just whe n he performs all those acts which  \nare obligatory upon him, and refrains from all those things which are  \nforbidden to him. And the sign of being just

# 4. Transforming

After the data is loaded, you then need to process and transform your data before putting it into a storage system. These transformations include chunking, extracting metadata, and embedding each chunk. This is necessary to make sure that the data can be retrieved, and used optimally by the LLM.

An `IngestionPipeline` uses a concept of Transformations that are applied to input data. These Transformations are applied to your input data

Reference: [IngestionPipeline](https://docs.llamaindex.ai/en/stable/module_guides/loading/ingestion_pipeline/)

In [14]:
chunk_size = 512
overlap_percentage = 0.25
chunk_overlap = int(chunk_size * overlap_percentage)

print(f"Chunk Size: {chunk_size}, Overlap Percentage: {overlap_percentage}, Chunk Overlap: {chunk_overlap}")

Chunk Size: 512, Overlap Percentage: 0.25, Chunk Overlap: 128


In [15]:
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap, paragraph_separator="\n\n\n"),
        embed_model # OpenAIEmbedding
    ],
    vector_store=vectorstore.vector_store,
)

In [16]:
documents = pipeline.run(documents=documents)

In [17]:
print(f"Total Documents: {len(documents)}")

Total Documents: 387


In [18]:
for i, doc in enumerate(documents[:10]):
    print(f"Document {i + 1} Length: {len(doc.text)}")

Document 1 Length: 263
Document 2 Length: 1595
Document 3 Length: 958
Document 4 Length: 1069
Document 5 Length: 1501
Document 6 Length: 1514
Document 7 Length: 1471
Document 8 Length: 1382
Document 9 Length: 1494
Document 10 Length: 1553


Function to load and add a new book to vectorstore

In [19]:
def add_book(book_path: str):
    print(f"Loading book from path: {book_path}")
    new_book = SimpleDirectoryReader(input_files=[book_path], file_metadata=extract_epub_metadata).load_data()
    print(f"Loaded book with metadata: {new_book[0].metadata}")
    new_book = pipeline.run(documents=new_book)
    print("Book added successfully!")

In [19]:
add_book("./data/give_and_take.epub")

Loading book from path: ./data/give_and_take.epub


e:\Capelin\llama-index-test\.venv\Lib\site-packages\ebooklib\epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')
e:\Capelin\llama-index-test\.venv\Lib\site-packages\ebooklib\epub.py:1423: FutureWarning: This search incorrectly ignores the root element, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/xmlns:rootfile[@media-type]'
  for root_file in tree.findall('//xmlns:rootfile[@media-type]', namespaces={'xmlns': NAMESPACES['CONTAINERNS']}):


Loaded book with metadata: {'id': 'epub-b09eb4f758f54495a29b6b23763dffb3', 'title': 'Give and Tak', 'author': 'Unknown', 'language': 'en', 'description': '', 'type': 'epub'}
Book added successfully!


# 5. Performing Vector Search

In [20]:
query_gen_prompt = """You are an AI language model assistant specializing in query expansion. Your task is to generate {num_queries} diverse versions of the given user question. These variations will be used to retrieve relevant documents from a vector database, helping to overcome limitations of distance-based similarity search.

Original question: {query}

Instructions:
1. Create {num_queries} unique variations of the original question.
2. Ensure each variation maintains the core intent of the original question.
3. Use different phrasings, synonyms, or perspectives for each variation.
4. Consider potential context or implications not explicitly stated in the original question.
5. Avoid introducing new topics or drastically changing the meaning of the question.

Please provide your {num_queries} question variations, each on a new line:
"""

In [21]:
top_n = 20
num_queries = 5
question = "Who was Salman Al Farsi a son of?"

vector_retriever = vectorstore.index.as_retriever(similarity_top_k=top_n)

In [22]:
retriever = QueryFusionRetriever(
    [vector_retriever],
    similarity_top_k=top_n,
    num_queries=num_queries,  # set this to 1 to disable query generation
    mode="reciprocal_rerank",
    use_async=True,
    verbose=True,
    query_gen_prompt=query_gen_prompt,
    llm=llm_openai,
)

In [23]:
# it will genereate n - 1 queries since the original query is also included
nodes_with_scores = retriever.retrieve(question)

Generated queries:
1. What is the lineage of Salman Al Farsi?
2. From whom does Salman Al Farsi descend?
3. Who is the father of Salman Al Farsi?
4. Can you identify the parentage of Salman Al Farsi?


In [24]:
# Printing first document
print("------- BOOK INFO -------")
print(f"Book Title: {nodes_with_scores[0].metadata['title']}")
print(f"Book ID   : {nodes_with_scores[0].metadata['id']}")
print(f"Author    : {nodes_with_scores[0].metadata['author']}")

print("\n------- TEXT -------")
print(nodes_with_scores[0].text)

------- BOOK INFO -------
Book Title: Slavery from Islamic and Christian Perspectives
Book ID   : epub-210bb65e0c8c49fd924340664086419c
Author    : Allamah Sayyid Sa'eed Akhtar Rizvi - XKP

------- TEXT -------
Salman had put a cooking pot on fire. The
twofriends were talking when all of a sudden the pot tumbled down and
overturned. But,wonder of wonders, not a single drop fell out of the pot.
Salman put the pot on the fireagain. After some time the same thing happened
again. No drop was spilt out, andSalman nonchalantly put it right again.

Abu Dharr was flabbergasted. At once he came out and met Imam 'Ali in the way.
Henarrated to him what he had seen. 'Ali said, "O Abu Dharr, if Salman informs
you ofall the things that he knows, you will wonder.O Abu Dharr, Salman is a
gate towards Allah on the earth. Anybody who accepts him is a believer,
anybody who rejects himis a kafir. Salman is from us - the family [of the
Prophet]."[44]

I think these few authentic traditions are enough to sh

In [25]:
nodes_with_scores[0].metadata

{'id': 'epub-210bb65e0c8c49fd924340664086419c',
 'title': 'Slavery from Islamic and Christian Perspectives',
 'author': "Allamah Sayyid Sa'eed Akhtar Rizvi - XKP",
 'language': 'en',
 'description': "Slavery is one of the oldest evils of society, Ancient civilizations could not eradicate slavery, so they compromised with it. Allamah Sayyid Sa'eed Akhtar Rizvi, the Chief Missionary of Bilal Muslim Mission, marshalled fact after fact from history; quoted from the Holy Qur'an, the traditions, and contemporary\nwriters on the subject; and cited Islamic and ancient laws. He has clearly and vividly\nshown that Western civilization is not so great a champion of emancipation of slaves\nas it poses to be. In fact this book will prove to he an eye-opener for those who\nblindly nod in approval to the propaganda about Western humanism.\n-\nISLAMICMOBILITY.COM\n-\nPublished by: Vancouver Islamic Educational Foundation British Columbia - Canada ISBN 0-920675-07-7",
 'type': 'epub',
 'embeddings': 'o

# 7. Generate Answer

In [26]:
answer_template = """You are a knowledgeable AI assistant tasked with answering questions based on the provided context. Your goal is to provide a comprehensive, accurate, and well-structured response using Chain-of-Thought reasoning.

Context:
{context_str}

Question: {query_str}

Instructions:
1. Carefully analyze the given context and question.
2. Use Chain-of-Thought reasoning to break down your answer into clear steps:
   a. First, identify the key components of the question, such as sub-problems that need to be explained before an answer can be derived
   b. Then, for each component, explain your thought process as you analyze the relevant information from the context.
   c. Show how you're connecting different pieces of information to form your conclusion.
3. Provide a detailed answer using only the information from the context.
4. If the context doesn't contain enough information to fully answer the question, state this clearly and explain why.
5. Organize your response with appropriate headings and subheadings for clarity.
6. Use bullet points or numbered lists where applicable to improve readability.
7. If relevant, include brief examples or analogies to illustrate key points.
8. After your detailed Chain-of-Thought reasoning, summarize your main points at the end of the response.
9. At the end, list all the contexts used in your reasoning. After your response, add a "References" section where you list the full contexts that you used arrive at your answer. Provide as much detail as available from each context (e.g., book title, author, full text of the relevant contexts. For video sources, include the url to the video.

For the references, use the format:

# References:
(for each context:)
## Context Id: title
Context excerpt (print as it is)

Please format your entire response in markdown for optimal readability.
"""

answer_prompt = PromptTemplate(answer_template)
# answer_prompt = BasePromptTemplate(answer_template)

## Response Modes


In [27]:
class ResponseMode(str, Enum):
    """Response modes of the response builder (and synthesizer)."""

    REFINE = "refine"
    """
    Refine is an iterative way of generating a response.
    We first use the context in the first node, along with the query, to generate an \
    initial answer.
    We then pass this answer, the query, and the context of the second node as input \
    into a “refine prompt” to generate a refined answer. We refine through N-1 nodes, \
    where N is the total number of nodes.
    """

    COMPACT = "compact"
    """
    Compact and refine mode first combine text chunks into larger consolidated chunks \
    that more fully utilize the available context window, then refine answers \
    across them.
    This mode is faster than refine since we make fewer calls to the LLM.
    """

    SIMPLE_SUMMARIZE = "simple_summarize"
    """
    Merge all text chunks into one, and make a LLM call.
    This will fail if the merged text chunk exceeds the context window size.
    """

    TREE_SUMMARIZE = "tree_summarize"
    """
    Build a tree index over the set of candidate nodes, with a summary prompt seeded \
    with the query.
    The tree is built in a bottoms-up fashion, and in the end the root node is \
    returned as the response
    """

    GENERATION = "generation"
    """Ignore context, just use LLM to generate a response."""

    NO_TEXT = "no_text"
    """Return the retrieved context nodes, without synthesizing a final response."""

    CONTEXT_ONLY = "context_only"
    """Returns a concatenated string of all text chunks."""

    ACCUMULATE = "accumulate"
    """Synthesize a response for each text chunk, and then return the concatenation."""

    COMPACT_ACCUMULATE = "compact_accumulate"
    """
    Compact and accumulate mode first combine text chunks into larger consolidated \
    chunks that more fully utilize the available context window, then accumulate \
    answers for each of them and finally return the concatenation.
    This mode is faster than accumulate since we make fewer calls to the LLM.
    """

In [28]:
def generate_answer(question, documents, answer_prompt, response_mode, verbose):
    response_synthesizer = get_response_synthesizer(response_mode=response_mode, llm=llm_groq, text_qa_template=answer_prompt, verbose=verbose)

    response = response_synthesizer.synthesize(
        question, nodes=documents
    )

    return response

`REFINE`:

Refine is an iterative way of generating a response. We first use the context in the first node, along with the query, to generate an initial answer. We then pass this answer, the query, and the context of the second node as input into a “refine prompt” to generate a refined answer. We refine through N-1 nodes, where N is the total number of nodes.

In [19]:
answer_md = generate_answer(question=question, documents=nodes_with_scores, answer_prompt=answer_prompt, response_mode=ResponseMode.REFINE, verbose=True)

> Refine context: title: Our Philosophy
author: Martyr Mohammad B...
> Refine context: title: Salman Al Muhammadi 
author: Kamal al-Sy...
> Refine context: title: "The Great Muslim Scientist and Philosop...
> Refine context: title: ASK THOSE WHO KNOW
author: Sayed Muhamma...
> Refine context: title: The Life of Imam Hasan Al-'Askari
author...
> Refine context: title: Our Philosophy
author: Martyr Mohammad B...
> Refine context: title: Salman Al Muhammadi 
author: Kamal al-Sy...
> Refine context: title: The Voice of Human Justice (Sautu'l 'Ada...
> Refine context: title: ASK THOSE WHO KNOW
author: Sayed Muhamma...
> Refine context: title: The Life of Imam Hasan Al-'Askari
author...
> Refine context: title: Our Philosophy
author: Martyr Mohammad B...
> Refine context: title: Salman Al Muhammadi 
author: Kamal al-Sy...
> Refine context: title: "The Great Muslim Scientist and Philosop...
> Refine context: title: ASK THOSE WHO KNOW
author: Sayed Muhamma...
> Refine context: title: Rays from

KeyboardInterrupt: 

In [ ]:
print(answer_md)

Monotheism emphasizes the singularity of God, rejecting any notion of multiple deities or fragmentation within His being. It highlights the existence of a solitary ultimate source of power and creation in the universe, reinforcing the idea of a singular divine authority.


`COMPACT`:

Compact and refine mode first combine text chunks into larger consolidated chunks that more fully utilize the available context window, then refine answers across them. This mode is faster than refine since we make fewer calls to the LLM.

In [17]:
answer_md = generate_answer(question=question, documents=nodes_with_scores, answer_prompt=answer_prompt, response_mode=ResponseMode.COMPACT, verbose=True)

[INFO] Query: Why is there only one God?
----------


				************************
				* [INFO] Iteration 1/1 *
				************************


-------------------------------
| [INFO] QUESTION SENT TO LLM: |
-------------------------------
You are a knowledgeable AI assistant tasked with answering questions based on the provided context. Your goal is to provide a comprehensive, accurate, and well-structured response using Chain-of-Thought reasoning.

Context:
id: epub-24542662e8fa47f2b48246bcd9c6c2e9
title: Theological Instructions (Amuzish-e Aqa'id)
author: Muhammad Taqi Misbah Yazdi
language: en
description: 
type: epub
embeddings: openaiembeddings

5- Why can one not regard every group of creation to be created by its
particular god?

6- How would you criticise the assumption that the entire universe is created
by one God, and at the same time claim that there are many lords and
executers?

7- From where does the illusion of several gods come from, and how can it be
nullified?

8- W

In [18]:
print(answer_md)

# Answer:

## Key Components:
1. **Monotheism vs. Polytheism**: Understanding the difference between monotheism (belief in one God) and polytheism (belief in multiple gods).
2. **Arguments for Monotheism**: Exploring the reasons why monotheism, specifically the belief in one God, is upheld.
3. **Critique of Polytheistic Beliefs**: Analyzing the flaws in the assumption of multiple gods and how it contradicts the unity of creation.
4. **Oneness of Lordship and Creatorship**: Explaining why the belief in Divine authority for saints does not affect the oneness of God.

## Chain-of-Thought Reasoning:

### 1. Monotheism vs. Polytheism:
- **Monotheism**: The belief in one God, which is central to the concept of oneness and unity.
- **Polytheism**: The belief in multiple gods, which leads to a fragmented view of creation.

### 2. Arguments for Monotheism:
- **Negation of Plurality**: Monotheism emphasizes the unity of God and rejects the idea of multiple independent gods.
- **Negation of Compo

`SIMPLE_SUMMARIZE`:

Merge all text chunks into one, and make a LLM call. This will fail if the merged text chunk exceeds the context window size.

In [42]:
answer_md = generate_answer(question=question, documents=nodes_with_scores[:5], answer_prompt=answer_prompt, response_mode=ResponseMode.SIMPLE_SUMMARIZE, verbose=True)

In [43]:
print(answer_md)

<think>
Okay, so I need to figure out who Salman Al Farsi's father was. Let me start by looking through the provided context to find any mentions of Salman's parentage.

First, I'll scan through the text for any references to Salman's family. I see several passages about Salman, but most of them talk about his virtues, his role as a companion of the Prophet, and his status in Islam. There's a part where Salman describes himself to 'Umar, saying, "I am Salman, son of a servant of Allah." That seems important. It suggests that his father was a servant of God, but it doesn't give a specific name or background.

I also notice that Salman was known as "Salman al-Farsi," which indicates he was from Persia. However, the context doesn't provide details about his father's name or his family lineage beyond what Salman himself mentions. The focus is more on his spiritual and religious qualities rather than his genealogy.

So, putting it together, while the context doesn't give Salman's father's n

In [39]:
from IPython.display import Markdown, display

display(Markdown(answer_md.response))

<think>
Alright, I need to figure out who Salman Al Farsi was the son of. Let me go through the provided context step by step.

First, I'll look for any mentions of Salman's background. Scanning through the context, I see multiple sections talking about Salman. One part mentions he was the son of a Zoroastrian priest in the province of Fars. That seems relevant.

I should check if there are other details about his father. The context doesn't provide the name of his father, just the occupation and region. So, it's clear that his father was a priest, but no specific name is given.

I don't see any conflicting information elsewhere in the context. All references to Salman's origin point to his father being a Zoroastrian priest without naming him. Therefore, the answer is based on the information given about his father's profession and background.
</think>

# Who Was Salman Al Farsi a Son Of?

## Chain-of-Thought Reasoning

### Step 1: Identify the Key Components of the Question
The question asks about the parentage of Salman Al Farsi, specifically focusing on his father. To answer this, we need to locate information within the provided context that discusses Salman's background and family.

### Step 2: Analyze the Context for Relevant Information
Upon examining the context, we find multiple references to Salman Al Farsi. The most relevant section is:

> "Salman al-Farsi (the Persian). He was the son of a Zoroastrian priest in the province of Fars."

This excerpt directly addresses Salman's lineage, identifying his father's occupation and religious background.

### Step 3: Connect the Information to Form a Conclusion
From the context, it is clear that Salman Al Farsi was the son of a Zoroastrian priest. The text does not provide the name of his father, only his profession and the region they were from.

### Step 4: Verify for Additional Information
No additional details about Salman's father, such as his name or personal history, are provided in the context. The focus is on Salman's religious journey and his significance in Islam.

## Detailed Answer

Salman Al Farsi was the son of a Zoroastrian priest from the province of Fars. The context does not mention his father's name but highlights his paternal occupation and regional origin.

## Summary of Main Points
- Salman Al Farsi's father was a Zoroastrian priest.
- The specific name of Salman's father is not provided in the context.
- Salman's background is discussed in the context of his religious journey and contributions to Islam.

## References

### Context Id: epub-210bb65e0c8c49fd924340664086419c
#### Excerpt:
"Salman al-Farsi (the Persian). He was the son of a Zoroastrian priest in the province of Fars."

`TREE_SUMMARIZE`:

Build a tree index over the set of candidate nodes, with a summary prompt seeded with the query. The tree is built in a bottoms-up fashion, and in the end the root node is returned as the response

In [21]:
from llama_index.core.response_synthesizers import TreeSummarize

summarizer = TreeSummarize(verbose=True, summary_template=answer_prompt, llm=llm)

response = summarizer.get_response(question, str(nodes_with_scores))

[INFO] Query: Why is there only one God?
----------
4 text chunks after repacking


-------------------------------
| [INFO] QUESTION SENT TO LLM: |
-------------------------------
You are a knowledgeable AI assistant tasked with answering questions based on the provided context. Your goal is to provide a comprehensive, accurate, and well-structured response using Chain-of-Thought reasoning.

Context:
[

N

o

d

e

W

i

t

h

S

c

o

r

e

(

n

o

d

e

=

T

e

x

t

N

o

d

e

(

i

d

_

=

'

d

f

e

3

2

e

7

0

-

a

7

9

6

-

4

7

f

1

-

8

2

f

4

-

2

9

9

f

8

2

d

5

9

a

5

2

'

,

e

m

b

e

d

d

i

n

g

=

N

o

n

e

,

m

e

t

a

d

a

t

a

=

{

'

i

d

'

:

'

e

p

u

b

-

2

4

5

4

2

6

6

2

e

8

f

a

4

7

f

2

b

4

8

2

4

6

b

c

d

9

c

6

c

2

e

9

'

,

'

t

i

t

l

e

'

:

"

T

h

e

o

l

o

g

i

c

a

l

I

n

s

t

r

u

c

t

i

o

n

s

(

A

m

u

z

i

s

h

-

e

A

q

a

'

i

d

)

"

,

'

a

u

t

h

o

In [22]:
print(response)

# Answer:

## Key Components:
1. **Refutation of Multiple Gods:** Analyzing the argument against the existence of multiple gods.
2. **Unity of Divine Authority:** Understanding the concept of singular divine authority.
3. **Logical Inconsistency of Polytheism:** Exploring the logical inconsistencies of polytheistic beliefs.

## Chain-of-Thought Reasoning:

### 1. Refutation of Multiple Gods:
- The context presents a logical argument against the existence of multiple causes or administrators in the universe.
- It emphasizes the impossibility of several causes bestowing existence for an effect or multiple lords governing the universe.
- This refutation forms the basis for establishing the belief in one God as the ultimate creator and administrator.

### 2. Unity of Divine Authority:
- The text discusses the acceptance of divine authority without the presence of independent lordship or creatorship.
- It highlights the coherence of creation and the necessity of a singular cause for the int

`GENERATION`:

Ignore context, just use LLM to generate a response.

In [23]:
answer_md = generate_answer(question=question, documents=nodes_with_scores, answer_prompt=answer_prompt, response_mode=ResponseMode.GENERATION, verbose=True)

[INFO] Query: Why is there only one God?
----------


-------------------------------
| [INFO] QUESTION SENT TO LLM: |
-------------------------------
Why is there only one God?




[INFO] Response from LLM: 'In monotheistic religions such as Christianity, Islam, and Judaism, there is only one God because it is believed that there is only one ultimate, supreme being who is all-powerful, all-knowing, and all-present. This belief is based on the idea that there can only be one ultimate source of creation and existence. Additionally, monotheistic religions teach that God is indivisible and cannot be divided into multiple beings.'


In [24]:
print(answer_md)

In monotheistic religions such as Christianity, Islam, and Judaism, there is only one God because it is believed that there is only one ultimate, supreme being who is all-powerful, all-knowing, and all-present. This belief is based on the idea that there can only be one ultimate source of creation and existence. Additionally, monotheistic religions teach that God is indivisible and cannot be divided into multiple beings.


`NO_TEXT`:

Return the retrieved context nodes, without synthesizing a final response.

In [25]:
answer_md = generate_answer(question=question, documents=nodes_with_scores, answer_prompt=answer_prompt, response_mode=ResponseMode.NO_TEXT, verbose=True)
print(answer_md) # this will return nothing since the response mode is set to NO_TEXT

None


In [26]:
print(answer_md.source_nodes) # But this will return the source nodes

[NodeWithScore(node=TextNode(id_='dfe32e70-a796-47f1-82f4-299f82d59a52', embedding=None, metadata={'id': 'epub-24542662e8fa47f2b48246bcd9c6c2e9', 'title': "Theological Instructions (Amuzish-e Aqa'id)", 'author': 'Muhammad Taqi Misbah Yazdi', 'language': 'en', 'description': '', 'type': 'epub', 'embeddings': 'openaiembeddings'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='a68c1e50-47bc-467a-9589-0a84bf4d0357', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'id': 'epub-24542662e8fa47f2b48246bcd9c6c2e9', 'title': "Theological Instructions (Amuzish-e Aqa'id)", 'author': 'Muhammad Taqi Misbah Yazdi', 'language': 'en', 'description': '', 'type': 'epub', 'embeddings': 'openaiembeddings'}, hash='c31b0860b0e4f8

`CONTEXT_ONLY`:

Returns a concatenated string of all text chunks.

In [27]:
answer_md = generate_answer(question=question, documents=nodes_with_scores, answer_prompt=answer_prompt, response_mode=ResponseMode.CONTEXT_ONLY, verbose=True)
print(answer_md)

id: epub-24542662e8fa47f2b48246bcd9c6c2e9
title: Theological Instructions (Amuzish-e Aqa'id)
author: Muhammad Taqi Misbah Yazdi
language: en
description: 
type: epub
embeddings: openaiembeddings

5- Why can one not regard every group of creation to be created by its
particular god?

6- How would you criticise the assumption that the entire universe is created
by one God, and at the same time claim that there are many lords and
executers?

7- From where does the illusion of several gods come from, and how can it be
nullified?

8- Why does it make no difference to the oneness of lordship and creatorship,
if one believes in the Divine authority for the saints?

# Lesson 17: The Meaning Of The Oneness Of The God

## Introduction

The word monotheism from the lexical point of view means ‘oneness or unity’,
but in philosophy, scholastic theology, ethics _(akhlaq),_ and mysticism it
has been applied with various different meanings. However it gives the same
basic meaning in all of these scien

`ACCUMULATE`:

Synthesize a response for each text chunk, and then return the concatenation.

In [28]:
answer_md = generate_answer(question=question, documents=nodes_with_scores, answer_prompt=answer_prompt, response_mode=ResponseMode.ACCUMULATE, verbose=True)

[INFO] Query: 'Why is there only one God?'


				************************
				* [INFO] Iteration 1/5 *
				************************


-------------------------------
| [INFO] QUESTION SENT TO LLM: |
-------------------------------
You are a knowledgeable AI assistant tasked with answering questions based on the provided context. Your goal is to provide a comprehensive, accurate, and well-structured response using Chain-of-Thought reasoning.

Context:
id: epub-24542662e8fa47f2b48246bcd9c6c2e9
title: Theological Instructions (Amuzish-e Aqa'id)
author: Muhammad Taqi Misbah Yazdi
language: en
description: 
type: epub
embeddings: openaiembeddings

5- Why can one not regard every group of creation to be created by its
particular god?

6- How would you criticise the assumption that the entire universe is created
by one God, and at the same time claim that there are many lords and
executers?

7- From where does the illusion of several gods come from, and how can it be
nullified?

8- Why does i

In [29]:
print(answer_md)

Response 1: # Answer:

## Key Components of the Question:
1. Why is there only one God?

## Chain-of-Thought Reasoning:

### 1. The Negation of Plurality:
- The concept of monotheism involves believing in the unity of God and negating plurality and multiplicity outside of God's essence.
- This belief opposes polytheism, which involves believing in multiple independent gods.
- Monotheism asserts that there is only one God, and all other entities are not separate gods but creations of this singular divine being.

### 2. The Negation of Composition:
- Another aspect of monotheism is the belief in the oneness of God's essence, which is self-evident and not composed of parts.
- This concept rejects the idea of God being made up of actual or potential parts, emphasizing the indivisible nature of God.
- By understanding God as indivisible and self-evident, the idea of multiple gods or lords becomes nullified.

### 3. Unity of Lordship and Creatorship:
- Belief in the oneness of God also exten

`COMPACT_ACCUMULATE`:

Compact and accumulate mode first combine text chunks into larger consolidated chunks that more fully utilize the available context window, then accumulate answers for each of them and finally return the concatenation. This mode is faster than accumulate since we make fewer calls to the LLM.

In [30]:
answer_md = generate_answer(question=question, documents=nodes_with_scores, answer_prompt=answer_prompt, response_mode=ResponseMode.COMPACT_ACCUMULATE, verbose=True)

[INFO] Query: 'Why is there only one God?'


				************************
				* [INFO] Iteration 1/1 *
				************************


-------------------------------
| [INFO] QUESTION SENT TO LLM: |
-------------------------------
You are a knowledgeable AI assistant tasked with answering questions based on the provided context. Your goal is to provide a comprehensive, accurate, and well-structured response using Chain-of-Thought reasoning.

Context:
id: epub-24542662e8fa47f2b48246bcd9c6c2e9
title: Theological Instructions (Amuzish-e Aqa'id)
author: Muhammad Taqi Misbah Yazdi
language: en
description: 
type: epub
embeddings: openaiembeddings

5- Why can one not regard every group of creation to be created by its
particular god?

6- How would you criticise the assumption that the entire universe is created
by one God, and at the same time claim that there are many lords and
executers?

7- From where does the illusion of several gods come from, and how can it be
nullified?

8- Why does i

In [31]:
print(answer_md)

Response 1: # Answer:

## Key Components:
1. Understanding the concept of monotheism and the oneness of God.
2. Exploring the arguments against the existence of multiple gods.
3. Analyzing the relationship between lordship, creatorship, and the Divine authority for saints.

## Chain-of-Thought Reasoning:

### 1. Understanding Monotheism and the Oneness of God:
- **Definition of Monotheism:** Monotheism refers to the belief in the existence of only one God.
- **Negation of Plurality:** Monotheism involves the negation of plurality and multiplicity outside the essence of God.
- **Negation of Composition:** Monotheism also entails the belief that God's essence is self-evident and not composed of parts.

### 2. Arguments Against Multiple Gods:
- **Creation by Multiple Gods:** It is impossible for every phenomenon to be created by several gods as this would lead to a multitude of existents, contradicting the unity of creation.
- **Creation by Particular Gods:** Assuming each phenomenon is c